Imports

In [145]:
#Omid55
from cvxpy import *
import numpy as np
from numpy import linalg as LA
import pandas as pd
import math

In [88]:
def clean_it(dataset):
    TEAM_SIZE = 5
    # remove those teams with 3 members
    print(dataset.shape)
    dataset = np.delete(dataset,np.where(dataset == -1)[0],axis=0) 
    print(dataset.shape)
    
    # remove identical matches
    dataset = unique_rows(dataset)
    print(dataset.shape)
    
    # remove identical teams
    if dataset.shape[1] <= 2*TEAM_SIZE + 1:
        # just champion levels
        dataset = np.delete(dataset, np.where(np.sum(dataset[:,:TEAM_SIZE] - dataset[:,TEAM_SIZE:2*TEAM_SIZE],axis=1)==0),axis=0)
    else:
        # champion levels and members
        dataset = np.delete(dataset, np.where(np.sum(dataset[:,:TEAM_SIZE] - dataset[:,TEAM_SIZE:2*TEAM_SIZE] + 
          dataset[:,2*TEAM_SIZE:3*TEAM_SIZE] - dataset[:,3*TEAM_SIZE:-1], axis=1) == 0), axis=0)
    print(dataset.shape)
    
    return dataset

In [86]:
def unique_rows(A, return_index=False, return_inverse=False):
    """
    Similar to MATLAB's unique(A, 'rows'), this returns B, I, J
    where B is the unique rows of A and I and J satisfy
    A = B[J,:] and B = A[I,:]

    Returns I if return_index is True
    Returns J if return_inverse is True
    """
    A = np.require(A, requirements='C')
    assert A.ndim == 2, "array must be 2-dim'l"

    B = np.unique(A.view([('', A.dtype)]*A.shape[1]),
               return_index=return_index,
               return_inverse=return_inverse)

    if return_index or return_inverse:
        return (B[0].view(A.dtype).reshape((-1, A.shape[1]), order='C'),) \
            + B[1:]
    else:
        return B.view(A.dtype).reshape((-1, A.shape[1]), order='C')

In [177]:
def compute_difference_of_winners_from_losers(matches):
    diff = np.zeros(len(matches))
    for i, match in enumerate(matches):
        loser = match[:TEAM_SIZE]
        winner = match[TEAM_SIZE:]
        diff[i] = LA.norm(winner,2) - LA.norm(loser,2)
    
    print('positive percentage: ', 100*float(len(np.where(diff > 0)[0]))/len(diff))
    print('zero percentage: ', 100*float(len(np.where(diff == 0)[0]))/len(diff))
    print('negative percentage: ', float(100*len(np.where(diff < 0)[0]))/len(diff))
    
    return diff

Body

In [195]:
matches = np.genfromtxt('matches_reduced.csv', delimiter=',', skip_header=True)
TEAM_SIZE = 5
matches = clean_it(matches)

'''
diff = compute_difference_of_winners_from_losers(matches)
winner_minus_loser = matches[:,TEAM_SIZE:] - matches[:,:TEAM_SIZE]


#winner_minus_loser = np.delete(winner_minus_loser, np.where(np.sum(winner_minus_loser, axis=1) == 0), axis=0)
    
diff, index = unique_rows(diff, return_index=True)

sorted_diff = np.sort(diff)
c = np.where(sorted_diff == 0)[0]
center_index = c[math.floor(len(c)/2)]
LEN = len(np.where(diff<0)[0])
beg_diff = sorted_diff[center_index-LEN]
end_diff = sorted_diff[center_index+LEN]

sampled_indices = np.where((diff >= beg_diff) * (end_diff >= diff))[0]
print(len(sampled_indices), 'number of close matches is sampled.')
close_matches = matches[sampled_indices]

print('\n')
compute_difference_of_winners_from_losers(close_matches)
np.savetxt(
    'close_matches.csv',
    close_matches,
    fmt='%d',
    delimiter=',',
    newline='\n',  # new line character
    footer='',  # file footer
    comments='',  # character to use for comments
    header='team1_member1,team1_member2,team1_member3,team1_member4,team1_member5,'
        + 'team2_member1,team2_member2,team2_member3,team2_member4,team2_member5'
    )
    '''

(102684, 10)
(100618, 10)
(32707, 10)
(31978, 10)


"\ndiff = compute_difference_of_winners_from_losers(matches)\nwinner_minus_loser = matches[:,TEAM_SIZE:] - matches[:,:TEAM_SIZE]\n\n\n#winner_minus_loser = np.delete(winner_minus_loser, np.where(np.sum(winner_minus_loser, axis=1) == 0), axis=0)\n    \ndiff, index = unique_rows(diff, return_index=True)\n\nsorted_diff = np.sort(diff)\nc = np.where(sorted_diff == 0)[0]\ncenter_index = c[math.floor(len(c)/2)]\nLEN = len(np.where(diff<0)[0])\nbeg_diff = sorted_diff[center_index-LEN]\nend_diff = sorted_diff[center_index+LEN]\n\nsampled_indices = np.where((diff >= beg_diff) * (end_diff >= diff))[0]\nprint(len(sampled_indices), 'number of close matches is sampled.')\nclose_matches = matches[sampled_indices]\n\nprint('\n')\ncompute_difference_of_winners_from_losers(close_matches)\nnp.savetxt(\n    'close_matches.csv',\n    close_matches,\n    fmt='%d',\n    delimiter=',',\n    newline='\n',  # new line character\n    footer='',  # file footer\n    comments='',  # character to use for comments\n

In [222]:
def save_it_for_optimizer(matches, name, LEN):
    X = matches[:,TEAM_SIZE:] - matches[:,:TEAM_SIZE]    #diff = winner - loser
    print(X.shape)
    X = np.delete(X, np.where(np.sum(X, axis=1) == 0), axis=0)
    print(X.shape)
    X = unique_rows(X)
    print(X.shape)
    if LEN > 0:
        idx = np.random.choice(len(X), LEN, replace=False)
        print('before samples:', diff.shape[0])
        X = X[idx,:]
        print('now samples:', X.shape[0])
        
    np.savetxt(
        name,
        X,
        fmt='%d',
        delimiter=',',
        newline='\n',  # new line character
        footer='',  # file footer
        comments='',  # character to use for comments
        header='winner1_loser1,winner2_loser2,winner3_loser3,winner4_loser4,winner5_loser5'
        )
    
    print('Positive: ', 100*len(np.where(np.sum(X, axis=1)>0)[0]) / len(X))
    print('Zero: ', 100*len(np.where(np.sum(X, axis=1)==0)[0]) / len(X))
    print('Negative: ', 100*len(np.where(np.sum(X, axis=1)<0)[0]) / len(X))

In [223]:
X = save_it_for_optimizer(matches, 'winner_minus_loser.csv', -1)

(31978, 5)
(31978, 5)
(4513, 5)
Positive:  89.94017283403501
Zero:  0.0
Negative:  10.05982716596499


In [194]:
#save_it_for_optimizer(close_matches, 'all_winner_minus_loser_close.csv', -1)